
# Домашна задача 4 - ВБС <br>
# Изработил: Бојан Ристов - 211151

# **Чекор 1: Подготовка на околината**

1. Инсталирање потребни библиотеки

In [ ]:
pip install torch-geometric

In [ ]:
import pandas as pd
import numpy as np
import spacy
import networkx as nx
import torch
import torchvision
import torchaudio
from torch_geometric.data import Data
from sklearn.model_selection import train_test_split

2. Вчитување на податочниte множества

In [ ]:
movies = pd.read_csv("movies_metadata.csv")
ratings = pd.read_csv("ratings_small.csv")

In [ ]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    45466 non-null  object 
 1   adult                 45466 non-null  object 
 2   budget                45466 non-null  object 
 3   genres                45466 non-null  object 
 4   original_language     45455 non-null  object 
 5   title                 45460 non-null  object 
 6   overview              44507 non-null  object 
 7   popularity            45461 non-null  object 
 8   production_companies  45463 non-null  object 
 9   production_countries  45463 non-null  object 
 10  revenue               45460 non-null  float64
 11  runtime               45203 non-null  float64
 12  vote_average          45460 non-null  float64
 13  vote_count            45460 non-null  float64
dtypes: float64(4), object(10)
memory usage: 4.9+ MB


In [ ]:
movies.head()

,id,adult,budget,genres,original_language,title,overview,popularity,production_companies,production_countries,revenue,runtime,vote_average,vote_count
0,862,FALSE,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",373554033.0,81.0,7.7,5415.0
1,8844,FALSE,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",262797249.0,104.0,6.9,2413.0
2,15602,FALSE,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",0.0,101.0,6.5,92.0
3,31357,FALSE,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",81452156.0,127.0,6.1,34.0
4,11862,FALSE,0,"[{'id': 35, 'name': 'Comedy'}]",en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",76578911.0,106.0,5.7,173.0


In [ ]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   userId   100004 non-null  int64  
 1   movieId  100004 non-null  int64  
 2   rating   100004 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 2.3 MB


In [ ]:
ratings.head()

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


3. Чистење на податоците

In [ ]:
movies.dropna(subset=['id', 'overview'], inplace=True)
ratings.dropna(subset=['userId', 'movieId', 'rating'], inplace=True)

movies['id'] = pd.to_numeric(movies['id'], errors='coerce')
ratings['movieId'] = pd.to_numeric(ratings['movieId'], errors='coerce')

movies.drop_duplicates(subset=['id'], inplace=True)
ratings.drop_duplicates(subset=['userId', 'movieId'], inplace=True)

# **Чекор 2: Екстракција на тројки од текст**

1. Иницијализација на spaCy модел

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

2. Функција за екстракција на тројки

In [ ]:
def extract_triples(text):
    doc = nlp(text)
    triples = []
    for sent in doc.sents:
        for token in sent:
            if token.dep_ == "ROOT":
                subj = [w.text for w in token.lefts if w.dep_ == "nsubj"]
                obj = [w.text for w in token.rights if w.dep_ == "dobj"]
                if subj and obj:
                    triples.append((subj[0], token.text, obj[0]))
    return triples

3. Примена на функцијата за overview колона

In [ ]:
movies['triples'] = movies['overview'].apply(lambda x: extract_triples(x) if isinstance(x, str) else [])
print(movies[['title', 'triples']].head())

                         title  \
0                    Toy Story   
1                      Jumanji   
2             Grumpier Old Men   
3            Waiting to Exhale   
4  Father of the Bride Part II   

                                             triples  
0                                                 []  
1                             [(they, invite, Alan)]  
2  [(wedding, reignites, feud), (divorcée, opens,...  
3    [(women, holding, breath), (Friends, talk, it)]  
4                             [(he, receives, news)]  


# **Чекор 3: Креирање на хетероген граф**

1. Инсталација на NetworkX

In [ ]:
pip install networkx

2. Креирање граф со јазли и врски

In [ ]:
import networkx as nx

G = nx.DiGraph()

for _, row in movies.iterrows():
    G.add_node(row['id'], type='movie',
               title=row['title'],
               genres=row['genres'],
               budget=row['budget'],
               language=row['original_language'])

for user_id in ratings['userId'].unique():
    G.add_node(user_id, type='user')

for _, row in ratings.iterrows():
    if row['rating'] >= 3:
        G.add_edge(row['userId'], row['movieId'], type='rated', rating=row['rating'])

# **Чекор 4: Модел за предвидување линкови**

In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.7 MB/s eta 0:00:00


1. Конверзија во PyTorch Geometric формат

In [ ]:
from torch_geometric.data import HeteroData
import torch

data = HeteroData()

movies[['budget', 'popularity', 'revenue', 'runtime', 'vote_average']] = movies[
    ['budget', 'popularity', 'revenue', 'runtime', 'vote_average']
].apply(pd.to_numeric, errors='coerce').fillna(0)

data['movie'].x = torch.tensor(
    movies[['budget', 'popularity', 'revenue', 'runtime', 'vote_average']].values, dtype=torch.float
)

data['user'].num_nodes = ratings['userId'].nunique()

edges = ratings[ratings['rating'] >= 3][['userId', 'movieId']].astype(int).values.T
data['user', 'rated', 'movie'].edge_index = torch.tensor(edges, dtype=torch.long)

print("Карактеристики на 'movie' јазлите:", data['movie'].x.shape)
print("Број на 'user' јазли:", data['user'].num_nodes)
print("Форма на edge_index:", data['user', 'rated', 'movie'].edge_index.shape)

Карактеристики на 'movie' јазлите: torch.Size([5, 5])
Број на 'user' јазли: 3
Форма на edge_index: torch.Size([2, 4])


2. Креирање GNN модел

In [ ]:
from torch_geometric.nn import SAGEConv

class LinkPredictionModel(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.lin = torch.nn.Linear(hidden_channels, 1)

    def forward(self, x_dict, edge_index_dict):
        x = self.conv1(x_dict, edge_index_dict)
        x = self.conv2(x, edge_index_dict)
        return self.lin(x['user'])

3. Тренирање и евалуација

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import HeteroConv, SAGEConv, Linear

class LinkPredictionModel(torch.nn.Module):
    def __init__(self, metadata, hidden_channels=64):
        super().__init__()
        self.convs = torch.nn.ModuleList()
        for edge_type in metadata[1]:
            conv = SAGEConv((-1, -1), hidden_channels)
            self.convs.append(conv)
        self.lin = Linear(hidden_channels, 1)

    def forward(self, x_dict, edge_index_dict):
        for i, (edge_type, edge_index) in enumerate(edge_index_dict.items()):
            x_dict[edge_type[0]] = self.convs[i](x_dict[edge_type[0]], edge_index)
        z = x_dict['user'] + x_dict['movie']
        return torch.sigmoid(self.lin(z))

model = LinkPredictionModel(metadata=data.metadata(), hidden_channels=64)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCELoss()

In [ ]:
for epoch in range(50):
    model.train()
    optimizer.zero_grad()
    predictions = model(data.x_dict, data.edge_index_dict)
    labels = torch.tensor([1, 0, 1, ...], dtype=torch.float)
    loss = criterion(predictions, labels)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

4. Модел за предвидување линкови - Втор начин

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import SAGEConv, to_hetero
from torch_geometric.data import HeteroData

# Проверка на податоците (примерни податоци)
data = HeteroData()
data["user"].x = torch.randn(100, 16)  # Примерни податоци за јазли
data["movie"].x = torch.randn(50, 16)
data["user", "rated", "movie"].edge_index = torch.randint(0, 100, (2, 300))
data["movie", "rated_by", "user"].edge_index = data["user", "rated", "movie"].edge_index.flip(0)

# Модел за предвидување линкови
class LinkPredictor(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv((in_channels, in_channels), hidden_channels)
        self.conv2 = SAGEConv((hidden_channels, hidden_channels), out_channels)

    def forward(self, x_dict, edge_index_dict):
        x_dict = {key: self.conv1(x, edge_index_dict[key]) for key, x in x_dict.items()}
        x_dict = {key: x.relu() for key, x in x_dict.items()}
        x_dict = {key: self.conv2(x, edge_index_dict[key]) for key, x in x_dict.items()}
        return x_dict

# Конверзија на моделот во хетерогена структура
model = LinkPredictor(16, 32, 16)
model = to_hetero(model, data.metadata())

# Оптимизатор и губиточна функција
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.BCEWithLogitsLoss()

# Тренирање
for epoch in range(50):
    model.train()
    optimizer.zero_grad()

    # Предвидувања
    pred = model(data.x_dict, data.edge_index_dict)

    # Генерирање на етикети за тренирање
    edge_label_index = data["user", "rated", "movie"].edge_index
    edge_labels = torch.ones(edge_label_index.size(1))  # Примерни позитивни етикети

    # Лос пресметка
    pred_edges = pred["user", "rated", "movie"]
    loss = criterion(pred_edges.view(-1), edge_labels)

    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

# Евалуација
model.eval()
with torch.no_grad():
    pred = model(data.x_dict, data.edge_index_dict)
    print("Evaluation predictions:", pred)

# **Чекор 5: Додавање тројки во граф**

1. Додавање на тројки како нови врски

In [ ]:
for _, row in movies.iterrows():
    for triple in row['triples']:
        subj, rel, obj = triple
        if obj not in G:
            G.add_node(obj, type='concept')
        G.add_edge(row['id'], obj, type=rel)

# **Чекор 6: Втор модел со дополнителни врски**

1. Додавање на нови јазли и врски во PyTorch Geometric

In [ ]:
new_edges = [...]
data['movie', 'related_to', 'concept'].edge_index = torch.tensor(new_edges, dtype=torch.long)

2. Креирање на мулти-тип ембединг модел

In [ ]:
class MultiEmbeddingModel(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.user_embed = torch.nn.Embedding(num_users, hidden_channels)
        self.concept_embed = torch.nn.Embedding(num_concepts, hidden_channels)
        self.conv1 = SAGEConv((-1, -1), hidden_channels)

    def forward(self, x_dict, edge_index_dict):
        x_dict['user'] = self.user_embed.weight
        x_dict['concept'] = self.concept_embed.weight
        x = self.conv1(x_dict, edge_index_dict)
        return x

3. Евалуација со ROC AUC

In [ ]:
predictions = model(data.x_dict, data.edge_index_dict)
auc = roc_auc_score(labels, predictions.detach().numpy())
print(f"ROC AUC: {auc}")